Домашнее задание к лекции "Базовые понятия статистики"
Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

## Задание 1

In [354]:
import numpy as np
import pandas as pd

In [355]:
data = pd.read_csv('horse_data.csv', sep=',', header=None, na_values='?')
data.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,2.0,1,530101,38.5,66.0,28.0,3.0,3.0,NaN,2.0,...,45.0,8.4,NaN,NaN,2.0,2,11300,0,0,2
1,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208,0,0,2
2,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,NaN,NaN,1.0,2,0,0,0,1
3,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1,2208,0,0,1
4,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300,0,0,2


In [356]:
data = data.drop(data.columns[[2, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27]], axis=1)
data.head()

,0,1,3,4,5,6,10,22
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


In [357]:
data.rename(columns = {0:'surgery', 1:'age', 3:'rectal_temperature', 4:'pulse', 5:'respiratory_rate', 
                       6:'temperature_of_extremities', 10:'pain', 22:'outcome'}, inplace = True)
data.head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


In [358]:
data.describe(include='all')

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


## Задание 2, 3

In [359]:
def remove_outliers (data, column):
    '''Функция принимает датафрейм и название нужного 
    столбца и удаляет строки с выбросами в этом столбце.
    Возвращает новый датафрейм без выбросов'''
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    new_data = data[data[column].between(lower_bound, upper_bound, inclusive=True)]
    return new_data

### Проанализируем каждый столбац
### 1. Хирургические операции

In [360]:
data['surgery'].value_counts()

1.0    180
2.0    119
Name: surgery, dtype: int64

Всего два значения 1 и 2. Поэтому, пропущенные значения заполним модой.

In [361]:
data['surgery'].fillna(data['surgery'].mode()[0], inplace=True)

### 2. Возраст

In [362]:
data['age'].value_counts()

1    276
9     24
Name: age, dtype: int64

Тоже всего два значения 1 и 9. В описании написано, что должны быть значения 1 и 2. Поэтому возможно, что 9 на самом деле может быть 2. Поэтому, заменим 9 значением 2.

In [363]:
data['age'].replace(9, 2, inplace=True)
data

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


### 3. Температура

Ищем выбросы

In [364]:
df_1 = remove_outliers(data, 'rectal_temperature')
df_1

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
292,1.0,1,37.0,66.0,20.0,1.0,4.0,2.0
294,2.0,1,38.5,40.0,16.0,1.0,2.0,1.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0


In [365]:
df_2 = pd.concat([data, df_1]).drop_duplicates(keep=False)
df_2[df_2['rectal_temperature'].notnull()].sort_values('rectal_temperature')

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0
141,2.0,1,36.0,42.0,30.0,NaN,NaN,2.0
238,2.0,1,36.1,88.0,NaN,3.0,3.0,3.0
80,1.0,1,36.4,98.0,35.0,3.0,4.0,2.0
118,1.0,1,36.5,78.0,30.0,1.0,5.0,1.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0
251,2.0,1,36.6,42.0,18.0,3.0,1.0,2.0
99,2.0,1,39.6,108.0,51.0,3.0,2.0,1.0
75,1.0,2,39.7,100.0,NaN,3.0,2.0,3.0
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0


Максимум и минимум температуры лежат в пределах возможного, поэтому не будем считать их выбросами.

In [366]:
data[data['rectal_temperature'].isnull()].head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
5,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0
7,1.0,1,NaN,60.0,NaN,3.0,NaN,2.0
8,2.0,1,NaN,80.0,36.0,3.0,4.0,3.0
16,1.0,2,NaN,128.0,36.0,3.0,4.0,2.0
28,1.0,1,NaN,NaN,NaN,NaN,NaN,2.0


Найдем показатели температуры в зависимости от возраста лошадей.

In [367]:
data.groupby('age')['rectal_temperature'].median()

age
1    38.1
2    38.6
Name: rectal_temperature, dtype: float64

In [368]:
data.groupby('age')['rectal_temperature'].mean()

age
1    38.124545
2    38.645000
Name: rectal_temperature, dtype: float64

Температуры для этих групп совсем незначительно отличаются, поэтому пустые значения заполним медианой по всей выборке.

In [369]:
data['rectal_temperature'].fillna(data['rectal_temperature'].median(), inplace=True)

### 4. Пульс

Ищем выбросы

In [370]:
df_3 = remove_outliers(data, 'pulse')
df_3

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
6,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
295,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [371]:
df_4 = pd.concat([data, df_3]).drop_duplicates(keep=False)
df_4[df_4['pulse'].notnull()].sort_values('pulse')

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
41,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
275,1.0,2,38.8,150.0,50.0,1.0,5.0,2.0
55,1.0,2,38.6,160.0,20.0,3.0,3.0,2.0
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
255,1.0,2,38.8,184.0,84.0,1.0,4.0,2.0


Выбросы лежат в пределах возможных значений, поэтому мы их оставим.

Найдем показатели пульса в зависимости от возраста.

In [372]:
data.groupby('age')['pulse'].mean()

age
1     67.456693
2    123.363636
Name: pulse, dtype: float64

In [373]:
data.groupby('age')['pulse'].median()

age
1     60.0
2    124.0
Name: pulse, dtype: float64

Показатели значительно разнятся в зависимости от возраста, поэтому пропуски заполним медианами для каждой группы отдельно.

In [374]:
data['pulse'].fillna(data.groupby('age')['pulse'].transform('median'), inplace=True)

### 5. Частота дыхания

Ищем выбросы

In [375]:
df_5 = remove_outliers(data, 'respiratory_rate')
df_5

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
6,1.0,1,37.9,48.0,16.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
294,2.0,1,38.5,40.0,16.0,1.0,2.0,1.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [376]:
df_6 = pd.concat([data, df_5]).drop_duplicates(keep=False)
df_6[df_6['respiratory_rate'].notnull()].sort_values('respiratory_rate')

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
120,1.0,1,39.4,54.0,66.0,1.0,2.0,1.0
103,1.0,2,38.0,140.0,68.0,1.0,3.0,1.0
125,1.0,1,38.0,42.0,68.0,4.0,3.0,1.0
265,2.0,1,38.2,56.0,68.0,3.0,3.0,3.0
229,1.0,2,38.5,120.0,70.0,NaN,NaN,1.0
295,1.0,1,38.2,120.0,70.0,4.0,2.0,3.0
41,2.0,2,39.0,150.0,72.0,NaN,NaN,1.0
82,1.0,2,38.1,100.0,80.0,3.0,3.0,1.0
84,1.0,1,37.8,60.0,80.0,1.0,2.0,1.0
208,1.0,1,37.8,88.0,80.0,3.0,NaN,3.0


Выбросы лежат в пределах возможных значений, поэтому мы их оставим.

Найдем показатели частоты дыхания в зависимости от возраста.

In [377]:
data.groupby('age')['respiratory_rate'].median()

age
1    24.0
2    49.0
Name: respiratory_rate, dtype: float64

Найдем показатели в зависимости от количества операций.

In [378]:
data.groupby('surgery')['respiratory_rate'].median()

surgery
1.0    28.0
2.0    24.0
Name: respiratory_rate, dtype: float64

Мы видим, что показатели частоты дыхания значительно отличаются в зависимости от возраста, поэтому пустые значения заполним медианами для каждой группы отдельно.

In [379]:
data['respiratory_rate'].fillna(data.groupby('age')['respiratory_rate'].transform('median'), inplace=True)

### 6. Температура конечностей

In [380]:
data['temperature_of_extremities'].value_counts()

3.0    109
1.0     78
2.0     30
4.0     27
Name: temperature_of_extremities, dtype: int64

Пропущенные значения заполним модой.

In [381]:
data['temperature_of_extremities'].fillna(data['temperature_of_extremities'].mode()[0], inplace=True)

### 7. Боль

In [382]:
data['pain'].value_counts()

3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64

Пропущенные значения заполним модой.

In [383]:
data['pain'].fillna(data['pain'].mode()[0], inplace=True)

### 8. Исход

In [384]:
data['outcome'].value_counts()

1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64

In [385]:
data[data['outcome'].isnull()]

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
132,1.0,1,38.0,48.0,20.0,3.0,4.0,NaN


Есть одно пустое значение. У этой лошади хорошие показатели, она молодая, поэтому сделаем предположение, что она выжила.

In [386]:
data['outcome'].fillna(1, inplace=True)

## Итоговый датафрейм

In [387]:
data.head()

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,3.0,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,3.0,3.0,2.0


In [388]:
data.describe(include='all')

,surgery,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,1.396667,1.080000,38.174333,71.386667,29.510000,2.470000,2.960000,1.550000
std,0.490023,0.271746,0.654831,27.968278,16.189613,0.975845,1.181689,0.736642
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,60.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000
